In [1]:
from datetime import date

from create_data import recreate_databases, create_tables
from jobs.order_created import do
from utils import get_spark_session

In [2]:
spark = get_spark_session()

2021-11-24 16:59:31.914 | INFO     | utils:get_spark_session:7 - Creating Spark Session


In [3]:
recreate_databases(spark)

2021-11-24 16:59:35.720 | INFO     | create_data:recreate_databases:29 - Removing spark warehouse (path = 'spark-warehouse')
2021-11-24 16:59:35.736 | INFO     | create_data:recreate_databases:33 - Creating database 'standardized_glovo_live'
2021-11-24 16:59:36.859 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_custom_events'
2021-11-24 16:59:36.880 | INFO     | create_data:recreate_databases:33 - Creating database 'mpcustomer_screen_views'
2021-11-24 16:59:36.903 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_custom_events'
2021-11-24 16:59:36.920 | INFO     | create_data:recreate_databases:33 - Creating database 'enriched_screen_views'


In [4]:
create_tables(spark)

2021-11-24 16:59:36.954 | INFO     | create_data:create_table:80 - Creating table 'standardized_glovo_live.cities'
2021-11-24 16:59:41.135 | INFO     | create_data:create_table:80 - Creating table 'standardized_glovo_live.devices'
2021-11-24 16:59:44.391 | INFO     | create_data:create_table:80 - Creating table 'standardized_glovo_live.orders'
2021-11-24 16:59:47.949 | INFO     | create_data:create_table:80 - Creating table 'mpcustomer_custom_events.order_created'


In [5]:
do(spark, exec_date=date(2021, 11, 19), n_days=3)

2021-11-24 16:59:51.565 | INFO     | adapters.loaders:load:30 - Loading 'mpcustomer_custom_events.order_created'
2021-11-24 16:59:51.632 | INFO     | adapters.loaders:load:30 - Loading 'standardized_glovo_live.cities'
2021-11-24 16:59:51.685 | INFO     | adapters.loaders:load:30 - Loading 'standardized_glovo_live.orders'
2021-11-24 16:59:51.706 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddTimezone'
2021-11-24 16:59:51.735 | INFO     | transformations.interfaces:_apply:13 - Applying transformation 'AddNumberOfOrders'
2021-11-24 16:59:51.900 | INFO     | adapters.writers:write:23 - Writting 'enriched_custom_events.order_created'


# Show input

In [6]:
spark.table("mpcustomer_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|
+-----------------------+-----------+-------------------+---------------+
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|
|                    CAG|          2|2021-11-01 07:29:00|     2021-11-01|
+-----------------------+-----------+-------------------+---------------+



# Show outputs

In [7]:
spark.table("enriched_custom_events.order_created").show()

+-----------------------+-----------+-------------------+---------------+-------------+------------+----------------+
|custom_attributes__city|customer_id|      creation_time|p_creation_date|    time_zone|country_code|number_of_orders|
+-----------------------+-----------+-------------------+---------------+-------------+------------+----------------+
|                    BCN|          1|2021-11-19 12:00:00|     2021-11-19|Europe/Madrid|          ES|               2|
|                    BCN|          1|2021-11-18 08:00:00|     2021-11-18|Europe/Madrid|          ES|               2|
|                    CAG|          2|2021-11-17 05:05:00|     2021-11-17|  Europe/Rome|          IT|               1|
+-----------------------+-----------+-------------------+---------------+-------------+------------+----------------+

